In [2]:
%load_ext autoreload
%autoreload 2
import os
import torch
import json
import yaml
import tqdm
from dataset.ConcatBaselineDataset import ConcatBaselineDataset
from baseline.models.ConcatBaselineNet import ConcatBaselineNet
from dataset.MurelNetDataset import MurelNetDataset
from murel.models.MurelNet import MurelNet
from baseline.scripts.train_baseline import get_hidden_layer_list
from torch.utils.data import DataLoader
import transforms.transforms as trfm

In [3]:
DATA_DIR = "/auto/homes/bat34/VQA/"
ROOT_DIR = "/auto/homes/bat34/VQA_PartII/"
MODEL_NAME = 'baseline'
CONFIG_PATH = os.path.join(ROOT_DIR, MODEL_NAME, 'scripts', MODEL_NAME + '.yaml')
with open(CONFIG_PATH) as f:
        config = yaml.load(f)
        
train_dataset = MurelNetDataset(split='train', \
                        txt_enc=config['txt_enc'], \
                        bottom_up_features_dir=config['bottom_up_features_dir'], \
                        skipthoughts_dir=config['skipthoughts_dir'], \
                        processed_dir=config['processed_dir'], \
                        ROOT_DIR=ROOT_DIR, \
                        vqa_dir=config['vqa_dir'])

test_dataset = MurelNetDataset(split='test', \
                        txt_enc=config['txt_enc'], \
                        bottom_up_features_dir=config['bottom_up_features_dir'], \
                        skipthoughts_dir=config['skipthoughts_dir'], \
                        processed_dir=config['processed_dir'], \
                        ROOT_DIR=ROOT_DIR, \
                        vqa_dir=config['vqa_dir'])


test_loader = DataLoader(test_dataset, shuffle=False, \
                         num_workers=config['num_workers'], \
                         batch_size=config['batch_size'], \
                         collate_fn=test_dataset.collate_fn, \
)
    
word_vocabulary = [word for word, _ in train_dataset.word_to_wid.items()]
#model = MurelNet(config, word_vocabulary)
model = ConcatBaselineNet(config, word_vocabulary)

BEST_MODEL_NAME = 'concat_baseline_self_attention_txt_enc_BayesianUniSkip_batch_size_256_lr_0.0001_lr_decay_rate_0.25_hidden_list_[1600, 1600]'
BEST_MODEL_PATH = os.path.join(ROOT_DIR, MODEL_NAME, 'trained_models', 'best_models', \
                             BEST_MODEL_NAME, 'best_model.pth')

#Path to model weights with best performance on the validation dataset
model.load_state_dict(torch.load(BEST_MODEL_PATH)['model'])
model = model.cuda()
model = model.eval()

/home/bat34/venv/lib/python3.5/site-packages/ipykernel_launcher.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [6]:
results = []
with torch.no_grad():
    for data in tqdm.tqdm(test_loader):
        item = {\
            'question_ids': data['question_ids'].cuda(), \
            'object_features_list': data['object_features_list'].cuda(), \
            'bounding_boxes': data['bounding_boxes'].cuda(), \
            'question_lengths': data['question_lengths'].cuda(), \
        }
        inputs = item
        qids = data['question_unique_id']
        outputs = model(inputs)
        values, ans_indices = torch.max(outputs, dim=1)
        ans_indices = list(ans_indices)
        ans_indices = [tsr.item() for tsr in ans_indices]
        for qid, ans_idx in zip(qids, ans_indices):
            results.append({
                'question_id': int(qid),
                'answer': test_dataset.aid_to_ans[ans_idx]
            })

100%|██████████| 1750/1750 [32:47<00:00,  1.12s/it] 


In [7]:
ID = 1
RESULTS_FILE_PATH = os.path.join(ROOT_DIR, MODEL_NAME, '{}_test-2015_results_{}.json'.format(MODEL_NAME, ID))
with open(RESULTS_FILE_PATH, 'w') as f:
    json.dump(results, f)